In [ ]:
# 3가지 타입(def read_type, art_type, view_type)으로 나뉘고 각각 실행해야함

# 4/30 ~ 4/1 1달간 총 14393개 *50개(3분)

## 입력값(이것만 입력할 것) ##
num = [0, 5827] # 수행할 구간, 매일경제은 range(0, n+1) 번 설정해야함, 내가 처음에 range(0,50)까지했으면 다음에는 range(50,이후숫자) 
loot = 'C:/Develops/newspapers/daily/' # 저장할 위치, 파일이 산만해지니 프로젝트가아닌폴더에서 관리할 것 # loot = './/' 현재위치에 저장하는 변수

# 실행할 함수 입력 -> 아래서 또 입력해줘야함
choice = 'read_type'

In [ ]:
#  파일이름
name = choice +'text' + '_' +  str(num[0]) + '_' + str(num[1]) # 파일이름 Ex. text_0_50

In [ ]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import time
import re

In [ ]:
df = pd.read_excel('.//NewsResult_20200901-20210430 (11).xlsx', names=['identical', 'date', 'press', 'name', 'title', 'c1', 'c2', 'c3', 'a1', 'a2', 'a3', 'person', 'place', 'institute', 'keyword', 'topkeyword', 'body', 'url', 'tf'])

In [ ]:
# 전처리
df = df[df.tf != '예외']
df = df[df.tf != '중복']
df = df[df.tf != '중복, 예외']
df = df[~df.title.str.contains('BOOK')]
# df = df[~df.title.str.contains('인터랙티브')]
# df = df[~df.place.str.contains('korea', na=False)]
# df = df[~df.place.str.contains('la', na=False)]
# df = df[~df.place.str.contains('LA', na=False)]
# df = df.reset_index()
# df = df.drop(columns=['index'], axis=1)

In [ ]:
len(df)

In [ ]:
# df.to_excel('.//pre_press_11.xls', header=None, index=False)

In [ ]:
df_read = df[df.url.str.contains('star')] # total 7133, ~4/1 5826
df_read = df_read.reset_index()
df_read = df_read.drop(columns=['index'], axis=1)

In [ ]:
df_read['url'][3]

In [ ]:
df_art = df[df.url.str.contains('news')] # total 10424, ~4/1 8424
df_art = df_art.reset_index()
df_art = df_art.drop(columns=['index'], axis=1)

In [ ]:
# df_art

In [ ]:
df = df[~df.url.str.contains('star')]
df = df[~df.url.str.contains('news')]

In [ ]:
df_view = df # rest_total 177 ~ 4/1 143
df_view = df_view.reset_index()
df_view = df_view.drop(columns=['index'], axis=1)

In [ ]:
# df_view

In [ ]:
# 셀레니움 창 오픈, 탭에 애드블락은 설치후 엑스해도 됨
options = webdriver.ChromeOptions()
options.add_extension(r'..//extension_3_11_1_0.crx')
browser = webdriver.Chrome('..//chromedriver.exe', options=options)

In [ ]:
def check_exists_by_xpath(xpath):
    try:
        browser.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [ ]:
# div.view_txt
# div.art_txt
# div.read_txt

In [ ]:
def read_type(num):
    texts = []
    for i in range(num[0], num[1]):
        try:
            url = df_read.url[i]
            browser.get(url)
            WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.XPATH, "//div[@class='read_txt']")))
            if check_exists_by_xpath("//div[@class='read_txt']") == True:
                html = browser.page_source
                soup = BeautifulSoup(html, 'html.parser')
                contents = soup.select('div.read_txt')
                text = ''
                for content in contents:
                    text = text + content.text.strip()
                texts.append(text)
            else:
                texts.append('None')
                continue
        except:
            texts.append('None')
            continue
    return texts

In [ ]:
def art_type(num):
    texts = []
    for i in range(num[0], num[1]):
        try:
            url = df_art.url[i]
            browser.get(url)
            WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.XPATH, "//div[@class='art_txt']")))
            if check_exists_by_xpath("//div[@class='art_txt']") == True:
                html = browser.page_source
                soup = BeautifulSoup(html, 'html.parser')
                contents = soup.select('div.art_txt')
                text = ''
                for content in contents:
                    text = text + content.text.strip()
                texts.append(text)
            else:
                texts.append('None')
                continue
        except:
            texts.append('None')
            continue
    return texts

In [ ]:
def view_type(num):
    texts = []
    for i in range(num[0], num[1]):
        try:
            url = df_art.url[i]
            browser.get(url)
            WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.XPATH, "//div[@itemprop='articleBody']")))
            if check_exists_by_xpath("//div[@itemprop='articleBody']") == True:
                html = browser.page_source
                soup = BeautifulSoup(html, 'html.parser')
                contents = soup.select('div[itemprop=articleBody]')
                text = ''
                for content in contents:
                    text = text + content.text.strip()
                texts.append(text)
            else:
                texts.append('None')
                continue
        except:
            texts.append('None')
            continue
    return texts

In [ ]:
# 실행할 함수 입력할 것
texts = read_type(num)

In [ ]:
len(texts)

In [ ]:
text = pd.DataFrame(data=texts) # texts를 text로 pandas로 저장

In [ ]:
spot = loot + name + '.xls'

In [ ]:
text.to_excel(spot, header=None,index=False)

In [ ]:
# 검증, 통합 설명과 수행방법(주석제거)

In [ ]:
# 분할된 text를 순서대로 합치는 법[2개(파일이름1, 파일이름2)를 불러울 경우]
# 파일이름 = pd.read_excel('C:/Develops/newspapers/press0/파일이름.xls', header=None) # 엑셀파일불러오기
# 파일이름 = 파일이름.values.tolist() # 리스트로 환원
# 파일이름2 = pd.read_excel('C:/Develops/newspapers/press0/파일이름2.xls', header=None)
# 파일이름2 = 파일이름2.values.tolist()
# co_text = 파일이름 + 파일이름2 # 리스트 순서대로 붙이기 

In [ ]:
# 제목과 내용 일치 여부 판단
# len(co_text)

In [ ]:
# i = 5 # co_text에 있는 숫자 구간에서 임의의 숫자 넣어서 제목과 내용이 일치하는지 대략적으로 검수
# df.title[i] # 번호에 해당하는 제목

In [ ]:
# co_text[i] # 번호에 해당하는 내용, co가 아니라 바로 결과를 확인할 때는 texts[i]

In [ ]:
# df에 text 넣어서 저장하기 # 모든 기사내용을 스크래핑해서 co_text로 합쳤으면 excel로 저장
# df['total_text'] = co_text
# final_spot = loot + 'press_0.xls'
# df.to_excel(final_spot, header=None,index=False)
# 끝